In [ ]:
#協程是指一個過程，與呼叫方協調作業，產出與呼叫方有關的結果
#呼叫方可以使用 .send() .close() .throw() 等方法

In [47]:
#協程的四種狀態: 'GEN_CREATED' 'GEN_RUNNING' 'GEN_SUSPENDED' 'GEN_CLOSED'
from inspect import getgeneratorstate
#getgeneratorstate(my_coroutine)

In [48]:
def simple_coroutine():
    print('start')
    x = yield #賦值這件事要等到右邊有產出後，才會發生
    print('received:',x)
    y = yield
    print('received:',y)

my_coro = simple_coroutine()

In [49]:
next(my_coro) #第一次用next啟動 #等同於my_coro.send(None)
#讓程序執行到第一個yield

start


In [50]:
my_coro.send('hi')
my_coro.send('ha')

received: hi
received: ha


StopIteration: 

In [16]:
my_coro #協同程序被定義成一種產生器函式

<generator object simple_coroutine at 0x000002E1177BF1C8>

In [27]:
def simple_coroutine2(a):
    print('start:',a)
    b = yield a #賦值這件事要等到右邊有產出後，才會發生
    print('received:',b)
    c = yield a+b
    print('received:',c)

my_coro2 = simple_coroutine2(1)

In [34]:
next(my_coro2)

start: 1


1

In [35]:
my_coro2.send(1.5)

received: 1.5


2.5

In [36]:
my_coro2.send(1000)

received: 1000


StopIteration: 

In [15]:
#移動平均協同程序

def avg():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

coro_avg = avg()
next(coro_avg)
coro_avg.send(10), coro_avg.send(30), coro_avg.send(10)

(10.0, 20.0, 16.666666666666668)

In [6]:
#avg()的問題是無法處理例外
coro_avg2 = avg()
next(coro_avg2)
coro_avg2.send(10), coro_avg2('spam')

NameError: name 'avg' is not defined

In [7]:
coro_avg2(20)#已無法使用

NameError: name 'coro_avg2' is not defined

In [8]:
#協同程序(產生器物件有兩個方法可以處理例外): throw 與 close

In [60]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [45]:
#正常啟動與關閉
from inspect import getgeneratorstate

coro_exc = demo_exc_handling()
next(coro_exc)
coro_exc.send(10)
coro_exc.send(20)
getgeneratorstate(coro_exc)

-> coroutine started
-> coroutine received: 10
-> coroutine received: 20


'GEN_SUSPENDED'

In [53]:
coro_exc.close()
getgeneratorstate(coro_exc)

'GEN_CLOSED'

In [64]:
#丟可處理例外 DemoException
from inspect import getgeneratorstate

coro_exc2 = demo_exc_handling()
next(coro_exc2)
coro_exc2.send(10)
coro_exc2.throw(DemoException)
getgeneratorstate(coro_exc2)

-> coroutine started
-> coroutine received: 10
*** DemoException handled. Continuing...


'GEN_SUSPENDED'

In [66]:
#丟不可處理的例外，程序會被終止
from inspect import getgeneratorstate

coro_exc3 = demo_exc_handling()
next(coro_exc3)
coro_exc3.send(20)
coro_exc3.throw(ZeroDivisionError)

-> coroutine started
-> coroutine received: 20


ZeroDivisionError: 

In [68]:
getgeneratorstate(coro_exc3)

'GEN_CLOSED'

In [21]:
#結束時有回傳值的avg3
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def avg_():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [25]:
coro_avg_ = avg_()
next(coro_avg_)
coro_avg_.send(5), coro_avg_.send(1)

(5.0, 3.0)

In [28]:
coro_avg_.send(None)

StopIteration: 